In [36]:
import cntk
import numpy
import cv2
from matplotlib import pyplot as plt
from PIL import Image

def create_reader(path, is_training, input_dim, num_label_classes):
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(features=featureStream))
    return cntk.io.MinibatchSource(deserailizer, randomize=is_training, max_sweeps=1, randomization_window_in_chunks=0)

def load_model(path):
    model = cntk.ops.functions.load_model(path)
    return model

def predict(model, data):
    return model.eval(data)

def do_predict(model_path, data_path, len_data):
    model = load_model(model_path)
    reader = create_reader(data_path, False, 3, 3)
    input_map = {
        input: reader.streams.features
    }
    data = reader.next_minibatch(len_data, input_map=input_map)
    
    feature_data = data[input].asarray()
    print("Finish Prediction")
    return predict(model, feature_data)

def img_to_txt(path):
    img = Image.open(path)
    pixel_list = []
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            pixel_list.append(img.getpixel((i,j)))
            
    pixel_set = set(pixel_list)
    count = 1
    with open("image_rgb_file.txt", 'w') as outfile:
        for p in pixel_set:
            outfile.write("|no " + str(count) + "|features " + str(p[0]) + " " + str(p[1]) + " " + str(p[2]) + "\n")
            count+=1
    print("Finish Convert Image")
    
    return pixel_set, len(pixel_set)

def data_to_img(predict_value, pixel_set, image_path, img_name):
    i = 0
    combined_dict = {}
    for e in pixel_set:
        #print(e, predict_value[i])
        combined_dict[e] = predict_value[i]
        i+=1
        
    #print(combined_dict[(0, 168, 246)][0])
    #print(type(combined_dict[(0, 168, 246)]))
    
    img = Image.open(image_path)
    pixels = img.load()
    
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            key = img.getpixel((i,j))
            #print(img.getpixel((i,j)), "=>" ,combined_dict[key])
            pixels[i,j] = (combined_dict[key][0], combined_dict[key][1], combined_dict[key][2])
            
    img.save(img_name)
    print("Finish Write Image")
    
def create_img(model_path, image_path):
    pixel_set, len_data = img_to_txt(image_path)
    data_path = "image_rgb_file.txt"
    predict_value = do_predict(model_path, data_path, len_data)
    predict_value = predict_value.round()
    data_to_img(predict_value, pixel_set, image_path, "output.png")
    
image_path = "Microsoft-Logo-3-small.jpg"
model_path = "./saved_model_exp_1/set-8.model"
create_img(model_path, image_path)

Finish Convert Image
Finish Prediction
54.0
Finish Write Image
